# Background field
Creation of a climatology using a background field create by `DIVAnd`. 

The key is in the definition of the _time selectors_:
1. for the background (using longer time periods) 
2. for the analysis.
Then in the final analysis, a `background` option allows user to specify the background file, the variable and the _time selector_.
```julia
...
background = DIVAnd.backgroundfile(filenamebackground,varname,TSbackground)
...
```

__Note:__ if you use a background field, don't use logarithmic transformations, which would let to error when applied to negative (anomaly) values.

In [ ]:
using DIVAnd
using CairoMakie
using GeoMakie
using NCDatasets
using PhysOcean
using DataStructures
using PyPlot
using Dates
using Statistics
using Random
using Printf
include("../config.jl")

## Files and directories
For this application we will work on the Baltic Sea.

In [ ]:
datafile = balticfile
varname = "salinity"

## Inputs
### Grid and depth levels

In [ ]:
Δx = 0.25
Δy = 0.25
lonr = 9.5:Δx:29.25
latr = 53.0:Δy:67.0
timerange = [Date(1900,1,1),Date(2020,12,31)];
depthr = [0.,5., 10., 15., 20., 30.];

### Data
Download if needed.

In [ ]:
download_check(balticfile, balticfileURL)

Read coordinates and observations

In [ ]:
@time obsval,obslon, obslat, obsdepth, obstime,obsid = DIVAnd.loadobs(Float64, datafile, varname);

Check the observation ranges:

In [ ]:
checkobs((obslon,obslat,obsdepth,obstime),obsval,obsid)

#### Quality check based on range

In [ ]:
sel = (obsval .<= 40) .& (obsval .>= 0);
obsval = obsval[sel]
obslon = obslon[sel]
obslat = obslat[sel]
obsdepth = obsdepth[sel]
obstime = obstime[sel]
obsid = obsid[sel];

#### Plot unique observation locations

In [ ]:
coords = [(x,y) for (x,y) in zip(obslon, obslat)];
coords_u = unique(coords);
@info "Total number of coordinates: $(length(coords))"
@info "Number of unique coordinates (profiles): $(length(coords_u))"
obslon_u = [x[1] for x in coords_u];
obslat_u = [x[2] for x in coords_u];

In [ ]:
f = GeoMakie.Figure()
ax = GeoAxis(f[1,1], title="Selected data points")
GeoMakie.scatter!(ax, obslon_u, obslat_u, color="black", markersize=1)
f

### Bathymetry

In [ ]:
bathname = gebco16file
download_check(gebco16file, gebco16fileURL)
bx, by, b = load_bath(bathname, true, lonr, latr);

#### Plotting

In [ ]:
f = GeoMakie.Figure()
ax = GeoAxis(f[1,1], title="Bathymetry")
GeoMakie.contourf!(ax, bx, by, b, levels = 20)
GeoMakie.contour!(ax, bx, by, b, levels = [0.], color="black")
f

#### Land-sea mask

In [ ]:
surfwater = b .>= depthr[1]
label = DIVAnd.floodfill(surfwater)
surfmask = label .== 1; # largest area has the label 1

In [ ]:
f = GeoMakie.Figure()
ax = GeoAxis(f[1,1], title="Land-sea mask")
GeoMakie.contourf!(ax, bx, by, surfmask, levels = 2, colormap=Reverse("binary"))
f

In [ ]:
mask = falses(size(b,1),size(b,2),length(depthr))
for k = 1:length(depthr)
    for j = 1:size(b,2)
        for i = 1:size(b,1)
            mask[i,j,k] = (b[i,j] >= depthr[k]) && surfmask[i,j]
        end
    end
end
@show size(mask)

## Analysis
### Background
For the background we create a _time selector_ with 4 seasons and 2 periods of years (1900-1989) and (1990-2017)

In [ ]:
yearlist = [1900:1989,1990:2017];
monthlist = [[1,2,3],[4,5,6],[7,8,9],[10,11,12]];  # Seasonal climatology
TSbackground = DIVAnd.TimeSelectorYearListMonthList(yearlist,monthlist);

#### Analysis parameters

In [ ]:
sz = (length(lonr), length(latr), length(depthr));
lenx = fill(200_000.,sz)   # 200 km
leny = fill(200_000.,sz)   # 200 km
lenz = [min(max(30.,depthr[k]/150),300.) for i = 1:sz[1], j = 1:sz[2], k = 1:sz[3]]
len = (lenx, leny, lenz);
epsilon2 = 0.1;

solver = :direct

filenamebackground = joinpath(outputdir, "Water_body_$(replace(varname," "=>"_"))_Baltic_background.4Danl.nc")
filename = joinpath(outputdir, "Water_body_$(replace(varname," "=>"_"))_Baltic.4Danl.nc")

#### Metadata for the netCDF file

Remove the background file if it already exists, and create the figure directory if needed.

In [ ]:
if isfile(filenamebackground)
    rm(filenamebackground) # delete the previous analysis
    @info "Removing file $filenamebackground"
end

Let's define a plotting function that well be applied at each time and depth level.

In [ ]:
aa = 3
lpad(aa, 3, '0')

In [ ]:
function plotres(timeindex, sel, fit, erri)
    tmp = copy(fit)
    nx,ny,nz = size(tmp)
    for i in 1:nz
        f = GeoMakie.Figure()
        ax = GeoAxis(f[1,1], title="Depth: $(depthr[i]) \n Time index: $(timeindex)")
        
        GeoMakie.ylims!(ax, minimum(latr) - Δy, maximum(latr) + Δy)
        GeoMakie.xlims!(ax, minimum(lonr) - Δx, maximum(lonr) + Δx)
        GeoMakie.contourf!(ax, bx, by, surfmask, levels = 2, colormap=Reverse("binary"))
        cont = GeoMakie.contourf!(ax, collect(lonr.-Δx/2.),collect(latr.-Δy/2.), tmp[:,:,i], levels=20, colormap=Reverse("RdYlBu"))

        cb = Colorbar(f[1, 2], cont)

        figname = varname * "_" * lpad(i, 2, '0') * "_" * lpad(timeindex, 3, '0') * ".png"
        GeoMakie.save(joinpath(figdir, figname), f)
    end
end

#### Background analysis
We now have all the input to prepare the background field with `diva3d`.     
The analysis has 8 times steps: 4 seasons times 2 periods (1900-1989 and 1990-2017).

In [ ]:
dbinfo = @time diva3d((lonr,latr,depthr,TSbackground),
           (obslon,obslat,obsdepth,obstime), obsval,
           len, epsilon2,
           filenamebackground,varname,
           bathname=bathname,
           mask = mask,
           fitcorrlen = false,
           niter_e = 2,
           solver = solver,
           MEMTOFIT = 120,
       );

### Final analysis
#### Parameters
We define the periods of interest for the final climatology.

In [ ]:
yearlist = [1970:1979,1980:1989,1990:1999,2000:2009];
monthlist = [[1,2,3],[4,5,6],[7,8,9],[10,11,12]];  # Seasonal climatology
TS = DIVAnd.TimeSelectorYearListMonthList(yearlist,monthlist);

We remove the final netCDF file if it exists.

In [ ]:
if isfile(filename)
    rm(filename) # delete the previous analysis
    @info "Removing file $filename"
end

Now we can perform the analysis using background field, defined through the option      
`background = DIVAnd.backgroundfile(filenamebackground,varname,TSbackground)`:      
* `filenamebackground` is the name of the netCDF that you have created with the background
* `varname` is the name of the variable as written in the background netCDF and
* `TSbackground` is the *TimeSelector` object used to compute the background.

With these options set, the new analysis will select the correct background according to the considered period.     
We now have 16 time steps: 4 seasons times 4 year periods.

In [ ]:
dbinfo = @time diva3d((lonr,latr,depthr,TS),
    (obslon,obslat,obsdepth,obstime), obsval,
    len, epsilon2,
    filename,varname,
    bathname=bathname,
    plotres = plotres,
    mask = mask,
    fitcorrlen = false,
    niter_e = 2,
    background = DIVAnd.backgroundfile(filenamebackground,varname,TSbackground),
    solver = solver,
    MEMTOFIT = 120,
);